## Preliminaries
Import Relevant Libraries

**Libraries for Machine Learning**

In [ ]:
from tensorflow.keras.applications import vgg19, resnet50, inception_resnet_v2, xception, mobilenet
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**Others**

In [ ]:
from fine_tuning_functions import build_train_model

## Generate Data
Using ImageDataGenerator

In [ ]:
train_dir = "../Output/train_test_split/ten/images/train"
test_dir = "../Output/train_test_split/ten/images/test"

datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.15,
    shear_range=0.15,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_batch_size = 128
test_batch_size = 32

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=train_batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=42
)
num_train = len(train_generator)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=test_batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=42
)
num_test = len(test_generator)

## Fine Tune

Load Pre-trained models

In [ ]:
vgg = vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
print("Loaded VGG19")

resnet = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
print("Loaded ResNet50")

xception_mod = xception.Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
print("Loaded Xception")

mobilenet_mod = mobilenet.MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
print("Loaded MobileNet")

inception_resnet = inception_resnet_v2.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
print("Loaded InceptionResNetV2")

In [ ]:
models = [vgg, resnet, xception_mod, mobilenet_mod, inception_resnet]

Freeze all but the last five layers (i.e. I will only update weights in the last five layers)

In [ ]:
for model in models:
    for layer in model.layers[:-5]:
        layer.trainable = False

Add new layers to pre-trained model and train with new data

In [ ]:
build_train_model(vgg, train_generator, test_generator, "VGG19")

In [ ]:
build_train_model(resnet, train_generator, test_generator, "ResNet50")

In [ ]:
build_train_model(xception_mod, train_generator, test_generator, "Xception")

In [ ]:
build_train_model(mobilenet_mod, train_generator, test_generator, "MobileNet")

In [ ]:
build_train_model(inception_resnet, train_generator, test_generator, "InceptionResNetV2")